In [3]:
import os
import nbformat as nbf
from nbformat.v4 import new_code_cell, new_markdown_cell
import warnings
from nbformat.validator import DuplicateCellId

def create_ipynb_from_repo(repo_path, output_filename):
    nb = nbf.v4.new_notebook()
    cells = []
    all_files = []

    # Collect all files and sort them according to the desired order
    for root, dirs, files in os.walk(repo_path):
        for file in files:
            if file.endswith('.md') or file.endswith('.py') or file.endswith('.ipynb'):
                all_files.append((root, file))

    all_files.sort(key=lambda x: (x[1].endswith('.md'), x[1].endswith('.ipynb'), x[1].endswith('.py')))

    for root, file in all_files:
        file_path = os.path.join(root, file)
        rel_path = os.path.relpath(file_path, repo_path)
        file_url = f"https://github.com/your_username/your_repo_name/blob/main/{rel_path}"

        cells.append(new_markdown_cell(f"## [{rel_path}]({file_url})"))

        with open(file_path, 'r') as f:
            file_content = f.read()
            if file.endswith('.md'):
                cells.append(new_markdown_cell(file_content))
            elif file.endswith('.ipynb'):
                nb_content = nbf.reads(file_content, as_version=4)
                cells.extend(nb_content['cells'])
            else:
                cells.append(new_code_cell(file_content))

    nb['cells'] = cells

    with open(output_filename, 'w') as f:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=DuplicateCellId)
            nbf.write(nb, f)

if __name__ == '__main__':
    repo_path = "."
    output_filename = "combined.ipynb"
    create_ipynb_from_repo(repo_path, output_filename)


C:\Users\todd_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\nbformat\__init__.py:129: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [ ]:
import os
import yaml

def create_yaml_from_repo(repo_path, output_filename):
    def add_to_dict_hierarchy(d, path_parts, file):
        if len(path_parts) == 1:
            d[path_parts[0]] = file
        else:
            if path_parts[0] not in d:
                d[path_parts[0]] = {}
            add_to_dict_hierarchy(d[path_parts[0]], path_parts[1:], file)

    repo_dict = {}
    
    for root, dirs, files in os.walk(repo_path):
        # Ignore .git directories
        if ".git" in root:
            continue
        
        for file in files:
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, repo_path)
            path_parts = rel_path.split(os.sep)

            add_to_dict_hierarchy(repo_dict, path_parts, file)

    with open(output_filename, 'w') as f:
        yaml.dump(repo_dict, f, sort_keys=True, indent=4)

if __name__ == '__main__':
    repo_path = "."
    output_filename = "repository_structure.yaml"
    create_yaml_from_repo(repo_path, output_filename)


In [2]:
import nbformat
from nbconvert import PDFExporter

def convert_ipynb_to_pdf(input_file, output_file):
    # Read the notebook
    with open(input_file, 'r') as f:
        nb_content = f.read()
    notebook = nbformat.reads(nb_content, as_version=4)

    # Convert the notebook to PDF
    pdf_exporter = PDFExporter()
    pdf_data, resources = pdf_exporter.from_notebook_node(notebook)

    # Write the PDF data to the output file
    with open(output_file, 'wb') as f:
        f.write(pdf_data)

if __name__ == "__main__":
    input_file = "combined.ipynb"
    output_file = "Digital_Twin_Guide.pdf"
    convert_ipynb_to_pdf(input_file, output_file)


OSError: xelatex not found on PATH, if you have not installed xelatex you may need to do so. Find further instructions at https://nbconvert.readthedocs.io/en/latest/install.html#installing-tex.

In [ ]:
!pip install --upgrade pip